<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/main/Basics_of_Knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.nn.utils.parametrize as P
from sklearn.datasets import load_iris
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import math


In [2]:
# Define a transform to normalize the data and flatten each image into a 1D array
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: x.view(-1))
])

def collate_fn(batch):
    data, target = zip(*batch)
    data = torch.stack(data)
    target = torch.tensor(target)
    return data.cuda(), target.cuda()

train_dataset = datasets.FashionMNIST('~/.data', download=True, train=True, transform=transform)
test_dataset = datasets.FashionMNIST('~/.data', download=True, train=False, transform=transform)
def split_dataset(dataset, labels):
    indices = [i for i, (data, label) in enumerate(dataset) if label in labels]
    return torch.utils.data.Subset(dataset, indices)

labels_1 = [0, 1, 2, 3, 4]
labels_2 = [5, 6, 7, 8, 9]

train_dataset_04 = split_dataset(train_dataset, labels_1)
train_dataset_59 = split_dataset(train_dataset, labels_2)

batch_size = 64

train_loader_1 = torch.utils.data.DataLoader(train_dataset_04, batch_size=batch_size, shuffle=True)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_59, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26421880/26421880 [00:01<00:00, 18791995.64it/s]


Extracting /root/.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 306636.26it/s]


Extracting /root/.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5567044.94it/s]


Extracting /root/.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12226657.41it/s]


Extracting /root/.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.data/FashionMNIST/raw



In [3]:
class FashionNet(nn.Module):
  def __init__(self, input_dims, num_class, input_hidden_size: 1000, hidden_layer_size:2000):

    super().__init__()
    self.input_dims = input_dims
    self.num_class = num_class
    self.input_hidden_size= input_hidden_size
    self.hidden_layer_size = hidden_layer_size


    self.input_layer = nn.Linear(in_features=self.input_dims, out_features=self.input_hidden_size)
    self.hidden_layer = nn.Linear(in_features=self.input_hidden_size, out_features=self.hidden_layer_size)
    self.output_layer = nn.Linear(in_features=self.hidden_layer_size, out_features=self.num_class)

  def forward(self, x):
    out = F.relu(self.input_layer(x))
    out = F.relu(self.hidden_layer(out))
    out = self.output_layer(out)
    return out

In [5]:
cuda_available = torch.cuda.is_available()

# Define the class names for Fashion MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

BigNet = FashionNet(input_dims=784, num_class=len(class_names), input_hidden_size=1000, hidden_layer_size=2000)
optimizer = optim.SGD(BigNet.parameters(), lr=0.0001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Assume `model` is your model
if cuda_available:
    BigNet = BigNet.to('cuda')
    optimizer = optim.SGD(BigNet.parameters(), lr=0.011, momentum=0.9)
    criterion = nn.CrossEntropyLoss().to('cuda')

is_on_gpu = next(BigNet.parameters()).is_cuda
print(f'Model is on GPU: {is_on_gpu}')

trainable_params = sum(p.numel() for p in BigNet.parameters() if p.requires_grad)
print(f'Number of trainable (tunable) parameters: {trainable_params}')

Model is on GPU: False
Number of trainable (tunable) parameters: 2807010


In [12]:
device='cpu'

import torch
from sklearn.metrics import confusion_matrix
import numpy as np

def train_model(epochs:int, train_loader, model, optimizer, criterion):
  model.train()
  train_loss_per_epoch = []

  for epoch in range(epochs):

    running_loss_train = 0.0

    for data_batch, target_batch in train_loader:

        data_batch = data_batch.to('cpu')
        target_batch = target_batch.to('cpu')

        optimizer.zero_grad()
        output = model(data_batch)
        loss_train = criterion(output, target_batch)
        loss_train.backward()
        optimizer.step()
        running_loss_train += loss_train.item()

    epcoh_loss_train = running_loss_train / len(train_loader)

    train_loss_per_epoch.append(epcoh_loss_train)


    if epoch %2 == 0:
      print(f'Epoch {epoch} Train loss {epcoh_loss_train}')
  return train_loss_per_epoch


def test(test_loader, net):
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in test_loader:
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x)
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                else:
                    wrong_counts[y[idx]] +=1
                total +=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the Label {i}: {wrong_counts[i]}')


In [19]:
# trainLoss = train_model(epochs=10,
#                         train_loader=train_loader_1,
#                         optimizer = optimizer,
#                         criterion=criterion,
#                         model=BigNet,
#                         )

testloss = test(test_loader=test_loader,
                        net=BigNet)

Accuracy: 0.437
wrong counts for the Label 0: 94
wrong counts for the Label 1: 47
wrong counts for the Label 2: 197
wrong counts for the Label 3: 126
wrong counts for the Label 4: 165
wrong counts for the Label 5: 1000
wrong counts for the Label 6: 1000
wrong counts for the Label 7: 1000
wrong counts for the Label 8: 1000
wrong counts for the Label 9: 1000


In [43]:
class LightFashionNet(nn.Module):
  def __init__(self, input_dims, num_class, input_hidden_size: 1000):

    super().__init__()
    self.input_dims = input_dims
    self.num_class = num_class
    self.input_hidden_size = input_hidden_size


    self.input_layer = nn.Linear(in_features=self.input_dims, out_features=self.input_hidden_size)
    self.output_layer = nn.Linear(in_features=self.input_hidden_size, out_features=self.num_class)

  def forward(self, x):
    out = F.relu(self.input_layer(x))
    out = self.output_layer(out)
    return out

In [45]:
LitNet = LightFashionNet(input_dims=784, input_hidden_size=1000, num_class=len(class_names))

In [46]:
# Print the norm of the first layer of the initial lightweight model
print("Norm of 1st layer of nn_light:", torch.norm(BigNet.input_layer.weight, p=2).item())
# Print the norm of the first layer of the new lightweight model
print("Norm of 1st layer of new_nn_light:", torch.norm(LitNet.input_layer.weight, p=2).item())

Norm of 1st layer of nn_light: 18.362703323364258
Norm of 1st layer of new_nn_light: 18.259302139282227


In [47]:
LitNet.input_layer.weight

Parameter containing:
tensor([[ 0.0234, -0.0337, -0.0106,  ...,  0.0163, -0.0135,  0.0131],
        [-0.0133,  0.0138,  0.0236,  ..., -0.0180,  0.0176,  0.0301],
        [-0.0200,  0.0197, -0.0341,  ..., -0.0216, -0.0162,  0.0121],
        ...,
        [ 0.0126,  0.0313, -0.0097,  ..., -0.0266,  0.0335,  0.0263],
        [ 0.0333,  0.0325,  0.0119,  ..., -0.0336, -0.0029, -0.0208],
        [-0.0088,  0.0121, -0.0318,  ..., -0.0183, -0.0084,  0.0319]],
       requires_grad=True)

In [48]:
total_params_deep = "{:,}".format(sum(p.numel() for p in BigNet.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in LitNet.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 2,807,010
LightNN parameters: 795,010


In [49]:
trainLoss = train_model(epochs=10,
                        train_loader=train_loader_1,
                        optimizer = optimizer,
                        criterion=criterion,
                        model=LitNet,
                        )

testloss = test(test_loader=test_loader,
                        net=LitNet)

Epoch 0 Train loss 2.2553637434424623
Epoch 2 Train loss 2.2553756562377343
Epoch 4 Train loss 2.2553685662080483
Epoch 6 Train loss 2.255374561240678
Epoch 8 Train loss 2.2553869757825122
Accuracy: 0.124
wrong counts for the Label 0: 988
wrong counts for the Label 1: 999
wrong counts for the Label 2: 146
wrong counts for the Label 3: 648
wrong counts for the Label 4: 1000
wrong counts for the Label 5: 1000
wrong counts for the Label 6: 1000
wrong counts for the Label 7: 1000
wrong counts for the Label 8: 979
wrong counts for the Label 9: 1000


In [67]:
def train_kd(teacher_model, student_model,  train_loader, epochs, criterion, T, device):

  optimizer = optim.Adam(student_model.parameters(), lr=0.0001)

  # set teacher model to eval mode
  teacher_model.eval()
  student_model.train()

  train_loss_per_epoch = []

  for epoch in range(epochs):
    running_loss_train = 0.0

    for data, target in train_loader:


      with torch.no_grad():
        logits_teacher = teacher_model(data)

      optimizer.zero_grad()
      student_logits = student_model(data)
      # Soften the student logits by applying softmax first and log() second

      soft_targets = nn.functional.softmax(logits_teacher / T, dim=-1)
      soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

      soft_targets_loss = -torch.sum(soft_targets * soft_prob) / soft_prob.size()[0] * (T**2)

      label_loss = criterion(student_logits, target)
      loss =  0.25*soft_targets_loss + 0.75*label_loss

      loss.backward()
      optimizer.step()
      running_loss_train+=loss.item()
      epcoh_loss_train = running_loss_train / len(train_loader)

    train_loss_per_epoch.append(epcoh_loss_train)

    if epoch %2 == 0:
      print(f'Epoch {epoch} Train loss {epcoh_loss_train}')
  return train_loss_per_epoch


In [68]:
train_kd(teacher_model=BigNet,
         student_model=LitNet,
         train_loader=train_loader_1,
         epochs=10, T= 1,
         device='cpu',
         criterion=nn.CrossEntropyLoss())

Epoch 0 Train loss 0.2986892391560174
Epoch 2 Train loss 0.28374231033233693
Epoch 4 Train loss 0.27360067988382475
Epoch 6 Train loss 0.26608492116302823
Epoch 8 Train loss 0.25810768062880296


[0.2986892391560174,
 0.2905925167903209,
 0.28374231033233693,
 0.27886686715553566,
 0.27360067988382475,
 0.269568753680949,
 0.26608492116302823,
 0.26121315055056166,
 0.25810768062880296,
 0.2543109116047176]

In [69]:
testloss = test(test_loader=test_loader,
                        net=LitNet)

Accuracy: 0.451
wrong counts for the Label 0: 87
wrong counts for the Label 1: 30
wrong counts for the Label 2: 161
wrong counts for the Label 3: 82
wrong counts for the Label 4: 129
wrong counts for the Label 5: 1000
wrong counts for the Label 6: 1000
wrong counts for the Label 7: 1000
wrong counts for the Label 8: 1000
wrong counts for the Label 9: 1000


In [70]:
train_kd(teacher_model=BigNet,
         student_model=LitNet,
         train_loader=train_loader_2,
         epochs=10, T= 1,
         device='cpu',
         criterion=nn.CrossEntropyLoss())

Epoch 0 Train loss 1.1719561132795013
Epoch 2 Train loss 0.9092024168226002
Epoch 4 Train loss 0.8917039465040032
Epoch 6 Train loss 0.88123553648178
Epoch 8 Train loss 0.873365246537906


[1.1719561132795013,
 0.9263361792828737,
 0.9092024168226002,
 0.8991554599326811,
 0.8917039465040032,
 0.8862633935169879,
 0.88123553648178,
 0.8771729211309063,
 0.873365246537906,
 0.8697663337182897]

In [71]:

testloss = test(test_loader=test_loader,
                        net=LitNet)

Accuracy: 0.594
wrong counts for the Label 0: 995
wrong counts for the Label 1: 122
wrong counts for the Label 2: 987
wrong counts for the Label 3: 777
wrong counts for the Label 4: 994
wrong counts for the Label 5: 53
wrong counts for the Label 6: 18
wrong counts for the Label 7: 43
wrong counts for the Label 8: 27
wrong counts for the Label 9: 44
